In [17]:
from pathlib import Path

import h5py
import pandas as pd
import numpy as np
import functools
import cudf, cuml, cupy
from cuml.feature_extraction.text import TfidfVectorizer
from cuml.neighbors import NearestNeighbors
print('RAPIDS',cuml.__version__)

RAPIDS 0.18.0


In [18]:
PATH = Path('/home/slex/data/shopee')

In [19]:
train_df = pd.read_csv(PATH/'train_split.csv')

In [20]:
train_gf = cudf.read_csv(PATH/'train_split.csv')
valid_gf = train_gf[train_gf.split==0]
print('valid shape is', valid_gf.shape )
valid_gf.head()

valid shape is (6808, 7)


,Unnamed: 0,posting_id,image,image_phash,title,label_group,split
6,6,train_1802986387,00144a49c56599d45354a1c28104c039.jpg,f815c9bb833ab4c8,Jubah anak size 1-12 thn,1835033137,0
7,7,train_1806152124,0014f61389cbaa687a58e38a97b6383d.jpg,eea7e1c0c04da33d,KULOT PLISKET SALUR /CANDY PLISKET /WISH KULOT...,1565741687,0
19,19,train_2961381387,00303ad1c062fdeaf5f41b9ffb71a5fb.jpg,e48d9b652098efe1,Madame Gie MakeUp Blush On BY GISELL,2098400894,0
23,23,train_1180155871,00369839c7d94411e54437b7cf59b603.jpg,bbc1c43ec3633ec0,[INCLUDE PAJAK] NCT DREAM RELOAD ALBUM,2224428568,0
24,24,train_1872347548,003906c6c6bba4cde15e5e854dba8f6d.jpg,f3e18c5ea14493e3,Stand Hanger Multifungsi - Tiang Gantungan Baj...,4044205497,0


In [21]:
tmp = train_df.groupby('label_group').posting_id.agg('unique').to_dict()
train_df['target'] = train_df.label_group.map(tmp)

In [22]:
model = TfidfVectorizer(stop_words='english', binary=True)
text_embeddings = model.fit_transform(valid_gf.title).toarray()
print('text embeddings shape is',text_embeddings.shape)

text embeddings shape is (6808, 9267)


In [23]:
KNN = 50
model = NearestNeighbors(n_neighbors=KNN)
model.fit(text_embeddings)
distances, indices = model.kneighbors(text_embeddings)

In [24]:
distances.shape

(6808, 50)

In [25]:
triplets=[]
for x in range(len(distances)):
    for y, val in zip(indices[x].tolist(), distances[x].tolist()):
        if val < 1.2: triplets.append((x,y,val))

In [26]:
triplets =sorted(triplets, key=lambda x: x[2])

In [27]:
triplets = triplets[:60000]

In [28]:
indices_x = np.array([x[0] for x in triplets])
indices_y = np.array([x[1] for x in triplets])
vals = np.array([x[2] for x in triplets])

In [30]:
# f = h5py.File("tfid_pairs.hdf5", "w")

# f.create_dataset('indices_x', data =indices_x)
# f.create_dataset('indices_y', data =indices_y)
# f.create_dataset('values', data =vals)
# f.close()

In [31]:
def groups(distances, indices, thresh):
    pred_groups = []
    for dists, inds in zip(distances, indices):
        gr=[]
        for d, i in zip(dists, inds):
            if d <= thresh:
                gr.append(i.item())
            else:
                break
        pred_groups.append(gr)
    return pred_groups

In [32]:
def f1score(row, col):
    val = row[col]
    if type(val)==str:val = eval(val)
    n = len( np.intersect1d(row.target,val) )
    return 2*n / (len(row.target)+len(val))


In [33]:
gr = groups(distances, indices, .8)
sum([len(g) for g in gr])/ len(gr)

2.45593419506463

In [34]:
matches = [train_df.iloc[g].posting_id.to_list() for g in gr]

In [263]:
train_df['matches_tfid'] = matches

In [264]:
train_df.to_csv('tfid_pred.csv')

In [265]:
image_df = pd.read_csv('resnet_arcface_matches08.csv')

In [266]:
valid_df = train_df[train_df.split==0].copy()

In [267]:
valid_df['matches_resnet'] = image_df['matches'].to_list()

In [268]:
def combine(row):
    a = eval(row.matches_resnet)
    b = row.matches_tfid
    return list(set(a+b))

In [269]:
valid_df['matches'] = valid_df.apply(combine, axis=1)

In [270]:
valid_df['score']=valid_df.apply(functools.partial(f1score, col='matches'),axis=1)

valid_df.score.mean()

0.7782010767540174

In [271]:
valid_df['score']=valid_df.apply(functools.partial(f1score, col='matches_tfid'),axis=1)

valid_df.score.mean()

0.6149885632935636

In [272]:
valid_df['score']=valid_df.apply(functools.partial(f1score, col='matches_resnet'),axis=1)

valid_df.score.mean()

0.7414798630474688

In [273]:
sum([len(g) for g in valid_df.matches])/len(valid_df)

5.978995299647473